In [2]:
import psycopg2
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris

In [3]:
# DB Connetion
db_connect = psycopg2.connect(
    user = "myuser",
    password= "mypassword",
    host= "localhost",
    port= 5432,
    database= "mydatabase"
)

In [4]:
# Data Insertion
X,y = load_iris(return_X_y=True, as_frame=True)
df = pd.concat([X,y], axis=1)

In [5]:
create_table_query = """
CREATE TABLE IF NOT EXISTS iris_data (
    id SERIAL PRIMARY KEY,
    timestamp timestamp,
    sepal_length float8,
    sepal_width float8,
    petal_length float8,
    petal_width float8,
    target int
);"""

In [6]:
with db_connect.cursor() as cur:
    cur.execute(create_table_query)
    db_connect.commit()

In [7]:
def create_table(db_connect) : 
    create_table_query = """
    CREATE TABLE IF NOT EXISTS iris_data (
        id SERIAL PRIMARY KEY,
        timestamp timestamp,
        sepal_length float8,
        sepal_width float8,
        petal_length float8,
        petal_width float8,
        target int
    );"""
    print(create_table_query)
    with db_connect.cursor() as cur:
        cur.execute(create_table_query)
        db_connect.commit()

# Data Insertion

In [8]:
# Data Insertion
X,y = load_iris(return_X_y=True, as_frame=True)
df = pd.concat([X,y], axis=1)
# db에 있는 테이블의 column과 맞춰주기
rename_rule = {
    "sepal length (cm)" : "sepal_length",
    "sepal width (cm)" : "sepal_width",
    "petal length (cm)" : "petal_length",
    "petal width (cm)" : "petal_width",
}

df = df.rename(columns=rename_rule)

In [9]:
df.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'target'], dtype='object')

In [10]:
# load 'iris data' -> rename
def get_data() : 
    X,y = load_iris(return_X_y=True, as_frame=True)
    df = pd.concat([X,y], axis = 1)
    origin_columns = list(df.columns)[:-1]
    new_columns = [(' ').join(col.split(' ', 2)[:-1]) for col in origin_columns]
    new_columns.append('target')
    df.columns = new_columns
    
    return df

In [13]:
# insertion query
# NOW() : timestamp
        
def insert_data(db_connect, data):
    insert_row_query = f"""
INSERT INTO iris_data
    (timestamp, sepal_length, sepal_width, petal_length, petal_width, target)
    VALUES (
        NOW(),
        {data.sepal_length},
        {data.sepal_width},
        {data.petal_length},
        {data.petal_width},
        {data.target});"""
        
    print(insert_row_query)
    with db_connect.cursor() as cur:
        cur.execute(insert_row_query)
        db_connect.commit()

# Data Insertion Loop

In [ ]:
import time

def generate_data(db_connect, df):
    while True:
        insert_data(db_connect, df.sample(1).squeeze())
        time.sleep(1)